In [ ]:
! pip install pypdf
! pip install langchain
! pip install pinecone-client
! pip install openai

In [14]:
from langchain.embeddings.openai import OpenAIEmbeddings

OPENAI_API_KEY = "API_KEY_HERE"
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

success


In [16]:
import pinecone

api_key = "API_KEY"
PINECONE_ENV = "us-west4-gcp-free"
index_name = 'INDEX_NAME'

pinecone.init(
    api_key=api_key,
    environment=PINECONE_ENV
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=1536  # 1536 dim of text-embedding-ada-002
    )

In [29]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

loader = PyPDFDirectoryLoader("books")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

In [ ]:
from langchain.vectorstores import Pinecone

testingIndex = Pinecone.from_documents(docs, embed, index_name=index_name)

In [ ]:
query = "What is mental accounting?"
testingIndex.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)[0].page_content.replace('\t', ' ')